In [24]:
from lark import Lark

In [25]:
from lark import Transformer

In [26]:
[1, 2, 3].pop(0)

1

In [31]:
class PathTransformer(Transformer):
    def identifier(self, args):
        assert len(args)==1
        return str(args[0])
    
    def slice(self, args):
        sargs = [None, None, None]
        i = 0
        for a in args:
            if a == ':':
                i+=1
            else:
                sargs[i] = int(a)
        return slice(*sargs)
    
    def int(self, args):
        return int(args[0])
    
    def complex(self, args):
        return complex(args[0])
    
    def float(self, args):
        return float(args[0])
    
    def none(self, args):
        return None
    
    def bool(self, args):
        return bool(args[0])
    
    def string(self, args):
        return args[0][1:-1]
    
    def other(self, args):
        return str(args[0])

    def path(self, args):
        return tuple(args)
    
    def tuple(self, args)


In [28]:
path_grammar = r'''
// A path is a series of dot-separated identifiers and [] based item-access.
path: identifier ("." identifier | "[" key "]")*

// item-access keys can be any hashable python literal
?key: element
    | tuple



?element: int
        | slice
        | float
        | complex
        | bool
        | none
        | string
        | other
        | tuple

tuple: "()"
     | "(" element ",)"
     | "(" element ("," element)+ [","] ")"
     

int.2: DEC_NUMBER
      | HEX_NUMBER
      | BIN_NUMBER
      | OCT_NUMBER

complex: [FLOAT_NUMBER "+"] IMAG_NUMBER
float: FLOAT_NUMBER

!slice.2: [int] ":"i [int] [":"i [int]]

identifier: /[a-zA-Z_]\w*/

string: /".*?(?<!\\)(\\\\)*?"/ | /'.*?(?<!\\)(\\\\)*?'/

none: "None"

!bool: "True" | "False"

other.0: /[^"'.\]\s]+/

DEC_NUMBER: /0|-?[1-9]\d*/i
HEX_NUMBER: /-?0x[\da-f]*/i
OCT_NUMBER: /-?0o[0-7]*/i
BIN_NUMBER : /-?0b[0-1]*/i
FLOAT_NUMBER: /-?((\d+\.\d*|\.\d+)(e[-+]?\d+)?|\d+(e[-+]?\d+))/i
IMAG_NUMBER: /-?\d+j/i | FLOAT_NUMBER "j"i
'''
p = Lark(path_grammar, start='path')


In [32]:
t = p.parse(r"my.foo[1].bar['baz'][True][1.7e-2][8other][:]")
t

Tree(path, [Tree(identifier, [Token(__ANON_2, 'my')]), Tree(identifier, [Token(__ANON_2, 'foo')]), Tree(int, [Token(DEC_NUMBER, '1')]), Tree(identifier, [Token(__ANON_2, 'bar')]), Tree(string, [Token(__ANON_4, "'baz'")]), Tree(bool, [Token(TRUE, 'True')]), Tree(float, [Token(FLOAT_NUMBER, '1.7e-2')]), Tree(other, [Token(__ANON_5, '8other')]), Tree(slice, [Token(COLON, ':')])])

In [33]:
PathTransformer().transform(t)

VisitError: Error trying to process rule "int":

int() argument must be a string, a bytes-like object or a number, not 'list'

In [23]:
PathTransformer().transform(p.parse(r"my.foo[(1,(2,3,()))]"))

('my', 'foo', (1, (2, 3, ())))